In [8]:
import time
import random
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

Определим формат ссылки для пагинации. Базовый url + параметр с номером страницы "?page=2"

In [9]:
BASE_URL = "https://lifehacker.ru"
TOPIC_URL = f"{BASE_URL}/topics/technology/"

BASE_URL, TOPIC_URL

('https://lifehacker.ru', 'https://lifehacker.ru/topics/technology/')

В разметке страницы статьи найдены уникальные классы и идентификаторы блока с названием и содержанием материала.

Заголовок берётся по классу article-card__title (h1),

Текст - из основного блока с id="article" берутся теги "p".

Сбор ссылок на статьи с первых 10 страниц рубрики

Для получения ссылок на материалы используем классы

Статьи находятся внутри контейнера article-card-container, где вложен блок div.container
Внутри него расположены карточки материалов с классом lh-small-article-card, а нужные ссылки на статьи размещены в тегах "a" с классом lh-small-article-card__link.

Для каждой страницы формируем корректный адрес: на первой странице используется базовая ссылка без параметров, а начиная со второй добавляется ?page=N. Затем загружаем HTML-код страницы через requests и разбираем его с помощью BeautifulSoup.
После этого внутри основного контейнера ищем все элементы "a" с нужным классом и извлекаем из них ссылки на статьи.
Каждая найденная ссылка преобразуется в абсолютную, чтобы получить полный URL.

In [10]:
all_links = []

for page in range(1, 11):  # страницы 1..10
    if page == 1:
        url = TOPIC_URL
    else:
        url = f"{TOPIC_URL}?page={page}"

    print(f"Страница рубрики {page}")
    print("URL:", url)

    # небольшая задержка, чтобы не спамить сервер
    time.sleep(random.uniform(1, 3))

    response = requests.get(url, timeout=10)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # основной контейнер со статьями
    container = soup.select_one("div.article-card-container div.container")
    if container is None:
        print("[WARN] article-card-container не найден, пропускаем страницу")
        continue

    page_links = []

    # внутри контейнера берём все карточки статей и ссылки по классам
    for a in container.select("div.lh-small-article-card a.lh-small-article-card__link"):
        href = a.get("href")
        if not href:
            continue

        full_url = urljoin(BASE_URL, href)
        page_links.append(full_url)

    print(f"Найдено ссылок на статьи на странице: {len(page_links)}")
    all_links.extend(page_links)

# Глобально убираем дубликаты, сохраняя порядок
unique_links = list(dict.fromkeys(all_links))
print("\nВсего уникальных ссылок на статьи:", len(unique_links))
unique_links[:5]

Страница рубрики 1
URL: https://lifehacker.ru/topics/technology/
Найдено ссылок на статьи на странице: 30
Страница рубрики 2
URL: https://lifehacker.ru/topics/technology/?page=2
Найдено ссылок на статьи на странице: 30
Страница рубрики 3
URL: https://lifehacker.ru/topics/technology/?page=3
Найдено ссылок на статьи на странице: 30
Страница рубрики 4
URL: https://lifehacker.ru/topics/technology/?page=4
Найдено ссылок на статьи на странице: 30
Страница рубрики 5
URL: https://lifehacker.ru/topics/technology/?page=5
Найдено ссылок на статьи на странице: 30
Страница рубрики 6
URL: https://lifehacker.ru/topics/technology/?page=6
Найдено ссылок на статьи на странице: 30
Страница рубрики 7
URL: https://lifehacker.ru/topics/technology/?page=7
Найдено ссылок на статьи на странице: 30
Страница рубрики 8
URL: https://lifehacker.ru/topics/technology/?page=8
Найдено ссылок на статьи на странице: 30
Страница рубрики 9
URL: https://lifehacker.ru/topics/technology/?page=9
Найдено ссылок на статьи на стр

['https://lifehacker.ru/xarakteristiki-galaxy-s26-ultra/',
 'https://lifehacker.ru/anons-modretro-m64/',
 'https://lifehacker.ru/igry-android-ios-noyabr-2025/',
 'https://lifehacker.ru/anons-black-shark-gs3-ultra/',
 'https://lifehacker.ru/zamedlenie-whatsapp/']

Перебрём все полученные ссылки и получим html-код каждого материала.
Распарсим полученный текст разметки с помощью BeautifulSoup, вытащив по каждой ссылке заголовок и содержание материала

Для каждой ссылки из unique_links:

1. Скачиваем HTML страницы статьи.
2. Берём h1 с классом "article-card__title" как заголовок.
3. Собираем все теги "p" и склеиваем их в один текст (через двойной перенос строки).

In [11]:
articles = []

total = len(unique_links)

for i, url in enumerate(unique_links, start=1):
    print(f"[{i}/{total}] Парсим статью: {url}")

    time.sleep(random.uniform(1, 3)) #тут делаем небольшую задержку, чтобы не блокировали запросы

    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        print("[ERROR] Не удалось скачать страницу:", e)
        continue

    soup = BeautifulSoup(resp.text, "html.parser")

    # Заголовок — h1 с классом article-card__title
    h1 = soup.find("h1", class_="article-card__title")
    if not h1:
        print("[WARN] Не найден тег <h1 class='article-card__title'>, пропускаем")
        continue
    title = h1.get_text(strip=True)

    # Основной текст — блок с id="article", внутри него <p>
    content_block = soup.find(id="article")
    if not content_block:
        print("[WARN] Не найден блок с id='article', пропускаем")
        continue

    paragraphs = content_block.find_all("p")
    text_parts = [p.get_text(" ", strip=True) for p in paragraphs]
    text_parts = [t for t in text_parts if t]
    text = "\n\n".join(text_parts)

    if not text.strip():
        print("[WARN] Пустой текст после парсинга, пропускаем")
        continue

    articles.append({
        "title": title,
        "text": text,
        "url": url,
    })

print("\nУспешно распарсено статей:", len(articles))


[1/300] Парсим статью: https://lifehacker.ru/xarakteristiki-galaxy-s26-ultra/
[2/300] Парсим статью: https://lifehacker.ru/anons-modretro-m64/
[3/300] Парсим статью: https://lifehacker.ru/igry-android-ios-noyabr-2025/
[4/300] Парсим статью: https://lifehacker.ru/anons-black-shark-gs3-ultra/
[5/300] Парсим статью: https://lifehacker.ru/zamedlenie-whatsapp/
[6/300] Парсим статью: https://lifehacker.ru/tysyachi-rekomendacii-po-naushnikam-s-reddit/
[7/300] Парсим статью: https://lifehacker.ru/prilozhenija-android-noyabr-2025/
[8/300] Парсим статью: https://lifehacker.ru/energoeffektivnyi-rezhim-v-google-kartax/
[9/300] Парсим статью: https://lifehacker.ru/20-gift-gadgets/
[10/300] Парсим статью: https://lifehacker.ru/mask-architects-solaris/
[11/300] Парсим статью: https://lifehacker.ru/google-ogranichila-besplatnyi-dostup-k-gemini-3-pro/
[12/300] Парсим статью: https://lifehacker.ru/anons-reetle-smartink-i/
[13/300] Парсим статью: https://lifehacker.ru/predstavlena-klaviatura-kernelcom/
[

Создаим датайфрейм с полученными данными


In [12]:
df = pd.DataFrame(articles)
df.head()

,title,text,url
0,Инсайдер сравнил характеристики смартфонов сер...,"Инсайдер Ice Universe, известный по утечкам о ...",https://lifehacker.ru/xarakteristiki-galaxy-s2...
1,Представлен прозрачный аналог Nintendo 64 с 4K...,Компания ModRetro представила ретро-консоль M6...,https://lifehacker.ru/anons-modretro-m64/
2,12 новых игр для Android и iOS: лучшее за ноябрь,Продолжаем дайджест мобильных новинок. В этом ...,https://lifehacker.ru/igry-android-ios-noyabr-...
3,Xiaomi показала сверхзащищённые часы Black Sha...,Xiaomi представила новые смарт-часы под брендо...,https://lifehacker.ru/anons-black-shark-gs3-ul...
4,В РКН подтвердили замедление WhatsApp: возможн...,Роскомнадзор подтвердил замедление работы What...,https://lifehacker.ru/zamedlenie-whatsapp/


In [13]:
output_path = "PZ_7_Volkov_lifehacker_technology_10_pages.csv"
df.to_csv(output_path, index=False)
print("Данные сохранены в файл:", output_path)

Данные сохранены в файл: PZ_7_Volkov_lifehacker_technology_10_pages.csv
